In [ ]:
from grid.model.navigation.objectinspection import ObjectInspect
from grid.model.navigation.visualservoing import VisualServoing
from grid.model.perception.detection.gdino import GroundingDINO
from grid.model.perception.detection.rt_detr import RT_DETR
from grid.model.perception.segmentation.sam2 import SAM2
from grid.model.perception.segmentation.gsam import GroundedSAM
from grid.model.perception.depth.midas import MIDAS
from grid.model.perception.depth.zoedepth import ZoeDepth
nav_inspect_0 = ObjectInspect()
nav_visualservoing_0 = VisualServoing()
detection_gdino_0 = GroundingDINO()
detection_rt_detr_0 = RT_DETR()
seg_sam2_0 = SAM2()
seg_gsam_0 = GroundedSAM()
depth_midas_0 = MIDAS()
depth_zoedepth_0 = ZoeDepth()

preprocessor_config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/172M [00:00<?, ?B/s]

final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# ===== Импорты для текущей работы =====

import airgen
import numpy as np
import rerun as rr
import time
from grid.model.navigation.visualservoing import VisualServoing

In [ ]:
# ===== Подключени к дрону взлет вертикальный =====

client = airgen.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)

# Взлёт
client.takeoffAsync().join()

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [ ]:
# ===== Маршрутизатор (ладно, таргетлок :З ) =====

# Установим точку назначения
start = client.getMultirotorState().kinematics_estimated.position
goal = airgen.Vector3r(10, 10, -5)  # пример точки B

print("Старт:", start)
print("Цель:", goal)


Старт: <Vector3r> {   'x_val': -7.126046057237545e-06,
    'y_val': -1.2493521808210062e-06,
    'z_val': -2.645627021789551}
Цель: <Vector3r> {   'x_val': 10,
    'y_val': 10,
    'z_val': -5}


In [ ]:
# ===== Получени данных Lidar + визуализация =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)

# Визуализация
rr.log('lidar/points', rr.Points3D(points))

""" Для аборигенов, в состоянии бпла, та залупа справа, отображается область действия лидара бпла, КАРОЧЕ ОБЛАКА ТОЧЕК ОКРУЖАЮЩИЕ БПЛА"""


' Для аборигенов, в состоянии бпла, та залупа справа, отображается область действия лидара бпла, КАРОЧЕ ОБЛАКА ТОЧЕК ОКРУЖАЮЩИЕ БПЛА'

In [ ]:
# ===== планировОЧКА маршрута и обход препятсвий =====

# ⛔ Заменяем airgen.simPlanPath заглушкой — вручную задаем точки маршрута
trajectory = [
    {"x_val": start.x_val, "y_val": start.y_val, "z_val": start.z_val},
    {"x_val": start.x_val + 5, "y_val": start.y_val, "z_val": start.z_val},
    {"x_val": goal.x_val, "y_val": goal.y_val, "z_val": goal.z_val}
]

# Преобразуем в список векторов
points = [airgen.Vector3r(p["x_val"], p["y_val"], p["z_val"]) for p in trajectory]

# Выполняем перемещение
velocity = 5.0
client.moveOnPathAsync(points, velocity, 120,
                       airgen.DrivetrainType.ForwardOnly,
                       airgen.YawMode(False, 0), -1, 0).join()
